In [1]:
# IMPORTANT: This part requires python 3 kernel
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'
from cycler import cycler

# fANOVA functional analysis
from fanova import fANOVA
import fanova.visualizer

# configuration space
from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformIntegerHyperparameter, UniformFloatHyperparameter

import numpy as np

import os

Using matplotlib backend: TkAgg


In [2]:
# scikit-optimize
import skopt
from skopt.plots import *

def objective(params):
    pass

In [3]:
def prepare_bayesopt_data(bayesOpt_res):
    """
    Synopsis
    ========
     Mise en forme des données retournées par la procédure d'optimisation
     Bayesienne afin d'être exploitées par dans le cadriciel fANOVA.
    
    Arguments
    =========
     BayesOpt_res   structure retournée par scikit-optimize
     
    Returns
    =======
     X  of shape (num_runs, num_hyperparams) containing in each row the
         values of the hyperparameters used in a particular run of the
         Bayesian optimization procedure.
     y  of shape (num_runs) containing the recognition performance of each
         run of the Bayesian optimization procedure with the corresponding
         set of hyper-parameters.
    """
    x_iters = bayesOpt_res.x_iters
    func_vals = bayesOpt_res.func_vals
    
    X = np.array([i for i in x_iters])
    y = np.array([i for i in func_vals])
    
    return X, y

In [4]:
def store(X, y, revision_name):
    folder_name = './fANOVA'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        
    np.savetxt(folder_name + '/X-' + revision_name + '.txt', X)
    np.savetxt(folder_name + '/y-' + revision_name + '.txt', y)

In [12]:
def build_fANOVA(revision_name, hyperparm_list, space):
    """
    Synopsis
    --------
    
    Arguments
    ---------
     revision_name   name of the current considered revision
     hyperparm_list  list of the hyper-parameters as used in the Bayesian
                      Optimisation procedure
     space           list of Hyperparameter objects
     
    Returns
    -------
     f               fANOVA object
     config_space    ConfigurationSpace object containing the Hyperparameter
                      objects
    """

    config_space = ConfigurationSpace()
    config_space.add_hyperparameters(space)
    
    # get the correspondance between hyperparams in space and config_space
    old_permutation = hyperparm_list
    permutation = config_space.get_hyperparameter_names()
    mapping = [old_permutation.index(val) for val in permutation]
    print(mapping)

    X = np.loadtxt('./fANOVA/X-' + revision_name + '.txt')
    y = np.loadtxt('./fANOVA/y-' + revision_name + '.txt')
    
    # Re-arrange features matrix X in order to match config_space.hyperparmeters order, using the mapping
    X = X[:, mapping]

    f = fANOVA(X, y, config_space=config_space)
    
    return f, config_space

In [8]:
hyperparm_list = [
    "n_hidden",
    "temperature",
]
space = [
    UniformIntegerHyperparameter("n_hidden", 1, 10, default_value=7),
    UniformFloatHyperparameter("temperature", 1e-1, 5e-1, default_value=0.5),
]

In [9]:
revision_name = 'acc2__'

In [14]:
res = skopt.load('../experiments/bayesOptResults.0.1.sav')
X, y = prepare_bayesopt_data(res)
store(X, y, revision_name)

In [15]:
folder_name = './fANOVA/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f, config_space = build_fANOVA(revision_name, hyperparm_list, space)
v = fanova.visualizer.Visualizer(f, config_space, folder_name)

[0, 1]


In [16]:
config_space.get_hyperparameter_names()

['n_hidden', 'temperature']

In [17]:
f.get_most_important_pairwise_marginals()

OrderedDict([(('n_hidden', 'temperature'), 0.06423761476177718)])

In [18]:
f.quantify_importance(('n_hidden', ))

{('n_hidden',): {'individual importance': 0.8887539745693009,
  'total importance': 0.8887539745693009,
  'individual std': 0.1496826592059107,
  'total std': 0.1496826592059107}}

In [19]:
v.create_all_plots()

INFO:fanova.visualizer.Visualizer:creating ./fANOVA/plots/acc2__/n_hidden.png
INFO:fanova.visualizer.Visualizer:creating ./fANOVA/plots/acc2__/temperature.png
INFO:fanova.visualizer.Visualizer:creating ./fANOVA/plots/acc2__/n_hidden_temperature.png
INFO:fanova.visualizer.Visualizer:creating ./fANOVA/plots/acc2__/interactive_plots


In [36]:
x = []
y = []
for hp in config_space.get_hyperparameter_names():
    s = f.quantify_importance((hp,))
    # print(s)
    for k, v in s.items():
        x.append(k[0])
        y.append(v['total importance'])

#colors = plt.cm.viridis(np.linspace(0, 1, 12))
#colors = [
#    colors[0]
#    , colors[2], colors[2], colors[2]
#    , colors[4], colors[4], colors[4]
#    , colors[6], colors[6], colors[6]
#    , colors[8], colors[8], colors[8]
#    , colors[10], colors[10], colors[10]
#    , colors[11], colors[11], colors[11]
#    , colors[9], colors[9], colors[9]
#    , colors[7]
#    , colors[5]
#    , colors[3]
#    , colors[1]
#]

N = 3
ind = np.arange(N)  # the x locations for the groups
width = 0.27       # the width of the bars

fig = plt.figure(figsize=(20, 5))
plt.bar(x, np.array(y)*100, log=True)  # , color=colors)
#plt.bar(x[0], y[0]*100, log=True, color='b')
#plt.bar(ind, y[1:4], log=True, color='r')
#plt.bar(ind+width, y[4:7], log=True, color='r')

# custumization
plt.xticks(rotation=70)
plt.ylabel('\% effect on performance variance')
plt.suptitle('Hyper-parameters importance over all experiments')
fig.savefig("./fANOVA-master/plots/all-in-all-allExperimentsAndBranches/hyperparamsImportance_overAllExperiments-Atempt2.svg", format="svg")

In [9]:
# %matplotlib
# plt = v.plot_pairwise_marginal([0, 4])
# plt.show()

In [10]:
# %matplotlib inline
# plt = v.plot_pairwise_marginal([2, 3])
# plt.show()

# Excluding parts of the parameters space where the prediction is not within the provided curoffs

In [4]:
revision_name = 'all-in-all-allExperimentsAndBranches'
folder_name = './fANOVA-master/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f, config_space = build_fANOVA(revision_name, hyperparm_list, space)
v = fanova.visualizer.Visualizer(f, config_space, folder_name)

[25, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 24, 23]


In [5]:
revision_name = 'all-in-all-allExperimentsAndBranches'
folder_name = './fANOVA-master/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f_quantile, config_space = build_fANOVA(revision_name, hyperparm_list, space)
f_quantile.set_cutoffs(quantile=(10,25))
v_bis = fanova.visualizer.Visualizer(f_quantile, config_space, folder_name)

[25, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 24, 23]


In [6]:
revision_name = 'all-in-all-allExperimentsAndBranches'
folder_name = './fANOVA-master/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f_cutoffs_1, config_space = build_fANOVA(revision_name, hyperparm_list, space)
f_cutoffs_1.set_cutoffs(cutoffs=(-np.inf,-0.50))
v_bis = fanova.visualizer.Visualizer(f_cutoffs_1, config_space, folder_name)

[25, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 24, 23]


In [7]:
revision_name = 'all-in-all-allExperimentsAndBranches'
folder_name = './fANOVA-master/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f_cutoffs_2, config_space = build_fANOVA(revision_name, hyperparm_list, space)
f_cutoffs_2.set_cutoffs(cutoffs=(-0.50, 0))
v_bis = fanova.visualizer.Visualizer(f_cutoffs_2, config_space, folder_name)

[25, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 24, 23]


In [8]:
revision_name = 'all-in-all-allExperimentsAndBranches'
folder_name = './fANOVA-master/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f_cutoffs_3, config_space = build_fANOVA(revision_name, hyperparm_list, space)
f_cutoffs_3.set_cutoffs(cutoffs=(-0.50, 0))
v_bis = fanova.visualizer.Visualizer(f_cutoffs_3, config_space, folder_name)

[25, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 24, 23]


In [9]:
revision_name = 'all-in-all-allExperimentsAndBranches'
folder_name = './fANOVA-master/plots/' + revision_name
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

f_cutoffs_4, config_space = build_fANOVA(revision_name, hyperparm_list, space)
f_cutoffs_4.set_cutoffs(cutoffs=(-0.50, 0))
v_bis = fanova.visualizer.Visualizer(f_cutoffs_4, config_space, folder_name)

[25, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 24, 23]


In [10]:
x = []
y = []
x_quantile = []
y_quantile = []
x_cutoffs_1 = []
y_cutoffs_1 = []
x_cutoffs_2 = []
y_cutoffs_2 = []
x_cutoffs_3 = []
y_cutoffs_3 = []
x_cutoffs_4 = []
y_cutoffs_4 = []

for hp in config_space.get_hyperparameter_names():
    s = f.quantify_importance((hp,))
    s_quantile = f_quantile.quantify_importance((hp,))
    s_cutoffs_1 = f_cutoffs_1.quantify_importance((hp,))
    s_cutoffs_2 = f_cutoffs_2.quantify_importance((hp,))
    s_cutoffs_3 = f_cutoffs_3.quantify_importance((hp,))
    s_cutoffs_4 = f_cutoffs_4.quantify_importance((hp,))
    # print(s)
    
    for k, v in s.items():
        x.append(k[0])
        y.append(v['total importance'])
    
    for k, v in s_quantile.items():
        x_quantile.append(k[0])
        y_quantile.append(v['total importance'])
    
    for k, v in s_cutoffs_1.items():
        x_cutoffs_1.append(k[0])
        y_cutoffs_1.append(v['total importance'])
        
    for k, v in s_cutoffs_2.items():
        x_cutoffs_2.append(k[0])
        y_cutoffs_2.append(v['total importance'])

    for k, v in s_cutoffs_3.items():
        x_cutoffs_3.append(k[0])
        y_cutoffs_3.append(v['total importance'])
        
    for k, v in s_cutoffs_4.items():
        x_cutoffs_4.append(k[0])
        y_cutoffs_4.append(v['total importance'])

In [8]:
N = 3
ind = np.arange(N)  # the x locations for the groups
width = 0.27       # the width of the bars

fig = plt.figure(figsize=(20, 5))
plt.bar(x, np.array(y)*100, log=True, color='b')
plt.bar(x, np.array(y_quantile)*100, log=True, bottom=np.array(y)*100, color='r')
plt.bar(x, np.array(y_cutoffs)*100, log=True, bottom=(np.array(y) + np.array(y_quantile))*100, color='g')

# custumization
plt.xticks(rotation=70)
plt.ylabel('\% effect on performance variance')
plt.suptitle('Hyper-parameters importance over all experiments')
fig.savefig("./fANOVA-master/plots/all-in-all-allExperimentsAndBranches/hyperparamsImportance_overAllExperiments-Atempt2.svg",
            format="svg")

In [33]:
N = 3
ind = np.arange(N)  # the x locations for the groups
width = 0.27       # the width of the bars

fig = plt.figure(figsize=(20, 5))
plt.grid(which='both', axis='y', linestyle='--', zorder=0)

plt.bar(x[1:22], np.array(y[1:22])*100, log=True)
#plt.bar(x, np.array(y_quantile)*100, log=True, bottom=np.array(y)*100, color='r')
plt.bar(x[1:22], np.array(y_cutoffs[1:22])*100, log=True, bottom=np.array(y[1:22])*100)

# custumization
plt.xticks(rotation=70)
plt.ylabel('\% effect on performance variance')
plt.suptitle('Hyper-parameters importance over all experiments')
fig.savefig("./fANOVA-master/plots/all-in-all-allExperimentsAndBranches/hyperparamsImportance_overAllExperiments-Atempt2-toto.svg",
            format="svg")

In [16]:
N = 3
ind = np.arange(N)  # the x locations for the groups
width = 0.27       # the width of the bars

fig = plt.figure(figsize=(20, 5))
plt.grid(which='both', axis='y', linestyle='--', zorder=0)

plt.bar(x[1:22], np.array(y[1:22])*100, log=True)
#plt.bar(x, np.array(y_quantile)*100, log=True, bottom=np.array(y)*100, color='r')
plt.bar(x[1:22], np.array(y_cutoffs_4[1:22])*100, log=True, bottom=np.array(y[1:22])*100)

# custumization
plt.xticks(rotation=70)
plt.ylabel('\% effect on performance variance')
plt.suptitle('Hyper-parameters importance over all experiments --- Models performing b/w 0.50 to np.inf')
fig.savefig("./fANOVA-master/plots/all-in-all-allExperimentsAndBranches/hyperparamsImportance_overAllExperiments-50-INF.svg",
            format="svg")